Importing all libraries

In [22]:
import numpy as np
import pandas as pd
from pandas import DataFrame, read_csv, to_numeric
from sklearn import cluster
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import math

import warnings
warnings.filterwarnings("ignore")

Loading all train and test data

In [23]:
train_data = (pd.read_csv('/media/anuja/study/Anuja/MS/CS_697A_ML/project/diabetes_train.csv',
                         sep=","))
train_data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
644,3,103,72,30,152,27.6,0.730,27,0
645,2,157,74,35,440,39.4,0.134,30,0
646,1,167,74,17,144,23.4,0.447,33,1
647,0,179,50,36,159,37.8,0.455,22,1


In [24]:
test_data = (pd.read_csv('/media/anuja/study/Anuja/MS/CS_697A_ML/project/diabetes_test.csv',
                         sep=","))
test_data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0,107,60,25,0,26.4,0.133,23,0
1,1,91,54,25,100,25.2,0.234,23,0
2,1,117,60,23,106,33.8,0.466,27,0
3,5,123,74,40,77,34.1,0.269,28,0
4,2,120,54,0,0,26.8,0.455,27,0
...,...,...,...,...,...,...,...,...,...
114,10,101,76,48,180,32.9,0.171,63,0
115,2,122,70,27,0,36.8,0.340,27,0
116,5,121,72,23,112,26.2,0.245,30,0
117,1,126,60,0,0,30.1,0.349,47,1


Q4: Use gradient boosting classifier with 5 different sets of parameters.

In [25]:
X_train = train_data.drop(columns = 'Outcome', axis =1)
Y_train = train_data.Outcome
print(X_train.shape)
print(Y_train.shape)
X_test = test_data.drop(columns = 'Outcome', axis =1)
Y_test = test_data.Outcome

(649, 8)
(649,)


In [39]:
# creating parameters set
from sklearn.ensemble import GradientBoostingClassifier
from itertools import product

def get_params():
    parameters = ({'loss': ['exponential'], 
                  'learning_rate':[0.1, 0.2],
                     'validation_fraction':[0.3],
                      'criterion':['friedman_mse','mse'],
                        'max_depth':[2, 5],})
    items = sorted(parameters.items())
    keys, values = zip(*items)
    for v in product(*values):
        params = dict(zip(keys, v))
        yield params
        
def gbc_model(params):
    gbc = GradientBoostingClassifier(**params)
    gbc.fit(X_train, Y_train)
    train_error = 1 - gbc.score(X_train, Y_train)
    test_error = 1 - gbc.score(X_test, Y_test)
    return train_error,test_error

params = get_params()
results = []
for p in params:
    gbc_train_error,gbc_test_error = gbc_model(p)
    results.append([p,gbc_train_error,gbc_test_error])

columns = ["Parameters","Train_error","Test_error"]
Result = pd.DataFrame(results,columns=columns)   
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(Result)
pd.set_option('display.max_colwidth', -1)
display(Result)

print("The best test error using GradientBoostingClassifier is : " + str(min(Result.Test_error)) )
print("\nThe best error for the parameters: ",Result.Parameters[5])
    

,Parameters,Train_error,Test_error
0,"{'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'exponential', 'max_depth': 2, 'validation_fraction': 0.3}",0.152542,0.235294
1,"{'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'exponential', 'max_depth': 5, 'validation_fraction': 0.3}",0.000000,0.218487
2,"{'criterion': 'friedman_mse', 'learning_rate': 0.2, 'loss': 'exponential', 'max_depth': 2, 'validation_fraction': 0.3}",0.117103,0.226891
3,"{'criterion': 'friedman_mse', 'learning_rate': 0.2, 'loss': 'exponential', 'max_depth': 5, 'validation_fraction': 0.3}",0.000000,0.235294
4,"{'criterion': 'mse', 'learning_rate': 0.1, 'loss': 'exponential', 'max_depth': 2, 'validation_fraction': 0.3}",0.152542,0.243697
5,"{'criterion': 'mse', 'learning_rate': 0.1, 'loss': 'exponential', 'max_depth': 5, 'validation_fraction': 0.3}",0.000000,0.210084
6,"{'criterion': 'mse', 'learning_rate': 0.2, 'loss': 'exponential', 'max_depth': 2, 'validation_fraction': 0.3}",0.117103,0.226891
7,"{'criterion': 'mse', 'learning_rate': 0.2, 'loss': 'exponential', 'max_depth': 5, 'validation_fraction': 0.3}",0.000000,0.243697


The best test error using GradientBoostingClassifier is : 0.2100840336134454

The best error for the parameters:  {'criterion': 'mse', 'learning_rate': 0.1, 'loss': 'exponential', 'max_depth': 5, 'validation_fraction': 0.3}


Q5: Combine the classifiers with the best test error you produced in Q1..Q4 using VotingClassifier and measure the training and test error for each of the following cases:  
a) give equal weight to each classifier 
b) give weight proportional to 1/(1+trainingerror) 